In [24]:
import numpy as np
import pandas as pd

In [31]:
df = pd.read_csv(r"online_retail_II.csv")
df.head()

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,is_free_item,is_ghost_item,is_return,is_cancel,total_price,price_bin,leak_amount,InvoiceDayOfWeek
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,False,False,False,False,83.4,£50 to < £200,0.0,Tuesday
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,False,False,False,False,81.0,£50 to < £200,0.0,Tuesday
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,False,False,False,False,81.0,£50 to < £200,0.0,Tuesday
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,False,False,False,False,100.8,£50 to < £200,0.0,Tuesday
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,False,False,False,False,30.0,£10 to < £50,0.0,Tuesday


In [33]:
df['revenue'] = df['total_price']  # positive only
gross_revenue = df[df['revenue'] > 0]['revenue'].sum()
gross_revenue


20476634.018

In [34]:
# Filter out zero-price items 
df_valid = df[df['price'] > 0].copy()

# Calculate total revenue loss from cancelled invoices
cancel_loss = df_valid[df_valid['is_cancel']]['total_price'].sum()

# Calculate total revenue loss from returns
return_loss = df_valid[df_valid['is_return']]['total_price'].sum()

# Put into a dictionary for clean output
revenue_leakage_kpis = {
    "cancel_loss": cancel_loss,
    "return_loss": return_loss,
    "total_measurable_revenue_loss": cancel_loss + return_loss
}

revenue_leakage_kpis


{'cancel_loss': -1462050.6099999999,
 'return_loss': -1462424.1800000002,
 'total_measurable_revenue_loss': -2924474.79}

In [32]:
free_items = df[df['price'] == 0]

free_items_stats = {
    "free_item_rows": len(free_items),
    "total_free_quantity": free_items['quantity'].sum(),
    "distinct_products": free_items['stockcode'].nunique(),
    "distinct_customers": free_items['customer_id'].nunique(),
}
free_items_stats


{'free_item_rows': 6014,
 'total_free_quantity': -318555,
 'distinct_products': 2971,
 'distinct_customers': 52}

In [35]:
print("Unique values in price column:", df['price'].unique()[:10])
print("Count where price == 0:", (df['price'] == 0).sum())
print("Count where price == '0':", (df['price'] == "0").sum())
print("Datatype of price:", df['price'].dtype)
print("Min price:", df['price'].min())
print("Number of rows with price < 0.1:", (df['price'] < 0.1).sum())



Unique values in price column: [6.95 6.75 2.1  1.25 1.65 5.95 2.55 3.75 5.45 1.69]
Count where price == 0: 6014
Count where price == '0': 0
Datatype of price: float64
Min price: -53594.36
Number of rows with price < 0.1: 6407


In [37]:
leakage_by_type = {
    "free_items": df[df['is_free_item']]['leak_amount'].sum(),
    "ghost_items": df[df['is_ghost_item']]['leak_amount'].sum(),
    "returns": df[df['is_return']]['leak_amount'].sum(),
    "cancellations": df[df['is_cancel']]['leak_amount'].sum()
}

pd.DataFrame.from_dict(leakage_by_type, orient='index', columns=['leak_amount'])



,leak_amount
free_items,0.00
ghost_items,0.00
returns,1462424.18
cancellations,1462797.75


In [41]:
product_loss = (
    df.groupby('stockcode')['leak_amount']
      .sum()
      .sort_values()
      .head(10)
)

product_loss



stockcode
23359     0.0
47504E    0.0
47504F    0.0
47518F    0.0
47518f    0.0
47554     0.0
47556     0.0
47556b    0.0
47559     0.0
47559b    0.0
Name: leak_amount, dtype: float64

In [42]:
customer_loss = (
    df.groupby('customer_id')['leak_amount']
      .sum()
      .sort_values()
      .head(10)
)

customer_loss


customer_id
18285    0.0
17058    0.0
17057    0.0
14913    0.0
17056    0.0
17055    0.0
14917    0.0
14918    0.0
14919    0.0
14920    0.0
Name: leak_amount, dtype: float64

In [43]:
country_loss = (
    df.groupby('country')['leak_amount']
      .sum()
      .sort_values()
      .head(10)
)

country_loss


country
West Indies            0.00
Lithuania              0.00
Iceland                0.00
Bermuda                0.00
Brazil                 0.00
Canada                 0.00
Thailand               0.00
European Community     8.50
Saudi Arabia          14.75
Lebanon               39.67
Name: leak_amount, dtype: float64